In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import r2_score
from tqdm import tqdm
import plotly.graph_objects as go

def preprocess_data(file_path, crypto_symbol):
    """
    Preprocess the cryptocurrency data with additional features (moving averages).
    """
    # Load data
    crypto_data = pd.read_csv(file_path)
    crypto_data['Date'] = pd.to_datetime(crypto_data['Date'])

    # Filter for specific cryptocurrency
    crypto_series = crypto_data[crypto_data['Symbol'] == crypto_symbol]

    # Calculate moving averages
    crypto_series['7-Day MA'] = crypto_series['Adj'].rolling(window=7).mean()
    crypto_series['14-Day MA'] = crypto_series['Adj'].rolling(window=14).mean()
    crypto_series['30-Day MA'] = crypto_series['Adj'].rolling(window=30).mean()

    # Prepare the data 
    crypto_series = crypto_series.dropna()  # Drop rows with NaN values 
    prophet_data = crypto_series[['Date', 'Adj', 'Volume', '7-Day MA', '14-Day MA', '30-Day MA']].rename(
        columns={'Date': 'ds', 'Adj': 'y'}
    )

    # Train-test split
    train_size = int(len(prophet_data) * 0.8)
    train = prophet_data[:train_size]
    test = prophet_data[train_size:]

    return train, test, crypto_series


def train_and_forecast_prophet(train, test, crypto_series):
    """
    Train Prophet model with additional features, make forecasts, and plot the results.
    """
    #  Prophet model with fine-tuned parameters
    print("Training the Prophet model...")
    prophet_model = Prophet(
        changepoint_prior_scale=0.1,
        seasonality_mode='multiplicative'
    )
    
    #  custom seasonality
    prophet_model.add_seasonality(name='weekly', period=7, fourier_order=3)
    prophet_model.add_seasonality(name='monthly', period=30.5, fourier_order=5)

    #  additional regressors
    prophet_model.add_regressor('Volume')
    prophet_model.add_regressor('7-Day MA')
    prophet_model.add_regressor('14-Day MA')
    prophet_model.add_regressor('30-Day MA')

    # Fit the model
    for _ in tqdm(range(1), desc="Training"):
        prophet_model.fit(train)

    # Create future dataframe for forecasting
    print("Creating future dataframe and forecasting...")
    future = test.copy() 
    forecast = prophet_model.predict(future)

    # Evaluate the model
    forecast_test = forecast[['ds', 'yhat']].iloc[-len(test):]
    actual_test = test['y'].reset_index(drop=True)
    r2 = r2_score(actual_test, forecast_test['yhat'])
    accuracy = r2 * 100  # Convert to percentage

    print(f"\nModel Accuracy: {accuracy:.2f}%")

    # Generate buy and sell signals
    buy_signals = crypto_series[(crypto_series['7-Day MA'] > crypto_series['30-Day MA']) &
                                (crypto_series['14-Day MA'] > crypto_series['30-Day MA'])]
    sell_signals = crypto_series[(crypto_series['7-Day MA'] < crypto_series['30-Day MA']) &
                                 (crypto_series['14-Day MA'] < crypto_series['30-Day MA'])]

    # Plot using Plotly
    fig = go.Figure()

    # candlestick chart
    fig.add_trace(go.Candlestick(
        x=crypto_series['Date'],
        open=crypto_series['Open'],
        high=crypto_series['High'],
        low=crypto_series['Low'],
        close=crypto_series['Adj'],
        name="OHLC"
    ))

    #  forecasted data
    fig.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat'],
        mode='lines',
        name='Forecasted Data',
        line=dict(color='orange', dash='dot')
    ))

    # moving averages
    fig.add_trace(go.Scatter(
        x=crypto_series['Date'],
        y=crypto_series['7-Day MA'],
        mode='lines',
        name='7-Day MA',
        line=dict(color='blue')
    ))
    fig.add_trace(go.Scatter(
        x=crypto_series['Date'],
        y=crypto_series['14-Day MA'],
        mode='lines',
        name='14-Day MA',
        line=dict(color='purple')
    ))
    fig.add_trace(go.Scatter(
        x=crypto_series['Date'],
        y=crypto_series['30-Day MA'],
        mode='lines',
        name='30-Day MA',
        line=dict(color='green')
    ))

    # buy signals
    fig.add_trace(go.Scatter(
        x=buy_signals['Date'],
        y=buy_signals['Adj'],
        mode='markers',
        name='Buy Signals',
        marker=dict(color='green', size=10, symbol='triangle-up')
    ))

    #  sell signals
    fig.add_trace(go.Scatter(
        x=sell_signals['Date'],
        y=sell_signals['Adj'],
        mode='markers',
        name='Sell Signals',
        marker=dict(color='red', size=10, symbol='triangle-down')
    ))

    
    fig.update_layout(
        title="Prophet Forecast with Buy/Sell Signals and Moving Averages",
        xaxis_title="Date",
        yaxis_title="Price (USD)",
        xaxis_rangeslider_visible=True,
        template="plotly_white",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    

    return accuracy, forecast, prophet_model,fig


